# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

<ipython-input-1-1499ab5789c7>:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
sys.path.append('/home/hiroki/research/dcase2021_task2/src/functions')
import common as com
import pytorch_modeler as modeler
from pytorch_model import ResNet38 as Model
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

## Setting

In [5]:
# Setting seed
modeler.set_seed(42)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_dir = INPUT_ROOT + "/dev_data"
add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = os.listdir(dev_dir)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/HDD1TB/research/dcase2021_task2/output/CNN_finetune/config.yaml'

In [7]:
torch.cuda.is_available()

True

## make path list

- __train_paths__ ([dict]): Each Machine Type
    - __ToyCar__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) :paths
    - __ToyTrain__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) : paths
    - ...

In [8]:
dev_paths = {}

for machine_type in machine_types:
    # dev train
    dev_train_paths = [f"{dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/train")]
    dev_train_paths = sorted(dev_train_paths)
    
    # add_dev train
    add_dev_paths = [f"{add_dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{add_dev_dir}/{machine_type}/train")]
    add_dev_paths = sorted(add_dev_paths)
    
    # dev_source valid
    dev_source_paths = [f"{dev_dir}/{machine_type}/source_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/source_test")]
    dev_source_paths = sorted(dev_source_paths)
    
    # dev_target valid
    dev_target_paths = [f"{dev_dir}/{machine_type}/target_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/target_test")]
    dev_target_paths = sorted(dev_target_paths)
    
    # bundle
    dev_paths[machine_type] = {}
    dev_paths[machine_type]['train'] = dev_train_paths + add_dev_paths
    dev_paths[machine_type]['valid_source'] = dev_source_paths
    dev_paths[machine_type]['valid_target'] = dev_target_paths

## training

In [9]:
#############################################################################
# run
#############################################################################
def run(machine_type, dev_paths):
    com.tic()
    
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dataloader
    dataloaders_dict = modeler.make_dataloader(dev_paths, machine_type)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    # model out path
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    logger.info('TRAINING')
    # parameter setting
    net = Model(sample_rate=config['param']['sample_rate'],
                window_size=config['param']['window_size'],
                hop_size=config['param']['hop_size'],
                mel_bins=config['param']['mel_bins'],
                fmin=config['param']['fmin'],
                fmax=config['param']['fmax'],
                classes_num=6
                )
    optimizer = optim.Adam(net.parameters())
    criterion = nn.CrossEntropyLoss()
    num_epochs = config['param']['num_epochs']
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e-2, 
                                              max_lr=1e-3, epochs=num_epochs, steps_per_epoch=len(dataloaders_dict['train']))
    # load pre-trained model
    pretrained_dict = torch.load(config['IO_OPTION']['PREMODEL_ROOT'])
    net.load_state_dict(pretrained_dict['model'], strict=False)
    # training
    output_dicts = modeler.train_net(net, dataloaders_dict, optimizer, criterion, scheduler, num_epochs, writer, model_out_path)
    
    com.toc()

In [10]:
machine_types

['fan', 'gearbox', 'pump', 'slider', 'ToyCar', 'ToyTrain', 'valve']

In [11]:
for machine_type in machine_types[5:]:
    run(machine_type, dev_paths)

2021-04-10 00:10:54,997 - 00_train.py - INFO - TARGET MACHINE_TYPE: ToyTrain
2021-04-10 00:10:54,998 - 00_train.py - INFO - MAKE DATA_LOADER
2021-04-10 00:10:55,040 - 00_train.py - INFO - TRAINING
2021-04-10 00:11:00,057 - pytorch_modeler.py - INFO - train
  0%|          | 0/189 [00:00<?, ?it/s]

use: cuda:0


100%|██████████| 189/189 [03:33<00:00,  1.13s/it]
2021-04-10 00:14:33,828 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 19/19 [00:07<00:00,  2.64it/s]
2021-04-10 00:14:43,067 - pytorch_modeler.py - INFO - Save best model
2021-04-10 00:14:43,067 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 19/19 [00:07<00:00,  2.60it/s]
2021-04-10 00:14:50,385 - pytorch_modeler.py - INFO - epoch:1/40, train_losses:26.193927757441998, val_source_losses:0.239036, val_target_losses:1.378628, best_flag:True
2021-04-10 00:14:50,386 - pytorch_modeler.py - INFO - train
100%|██████████| 189/189 [03:34<00:00,  1.13s/it]
2021-04-10 00:18:24,828 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 19/19 [00:07<00:00,  2.64it/s]
2021-04-10 00:18:33,872 - pytorch_modeler.py - INFO - Save best model
2021-04-10 00:18:33,873 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 19/19 [00:06<00:00,  2.80it/s]
2021-04-10 00:18:40,665 - pytorch_modeler.py - INFO - epoch:2/40, trai

100%|██████████| 19/19 [00:06<00:00,  2.86it/s]
2021-04-10 02:07:13,614 - pytorch_modeler.py - INFO - epoch:32/40, train_losses:3.560312606967045, val_source_losses:0.200237, val_target_losses:0.187385, best_flag:False
2021-04-10 02:07:13,615 - pytorch_modeler.py - INFO - train
100%|██████████| 189/189 [03:23<00:00,  1.08s/it]
2021-04-10 02:10:37,026 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 19/19 [00:06<00:00,  2.85it/s]
2021-04-10 02:10:43,688 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 19/19 [00:06<00:00,  2.85it/s]
2021-04-10 02:10:50,354 - pytorch_modeler.py - INFO - epoch:33/40, train_losses:3.7105947104032566, val_source_losses:0.199629, val_target_losses:0.195294, best_flag:False
2021-04-10 02:10:50,354 - pytorch_modeler.py - INFO - train
100%|██████████| 189/189 [03:23<00:00,  1.08s/it]
2021-04-10 02:14:13,761 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 19/19 [00:06<00:00,  2.86it/s]
2021-04-10 02:14:20,416 - pytorch_modeler.py

elapsed time: 8712.580429554 [sec]


...

use: cuda:0


100%|██████████| 189/189 [03:25<00:00,  1.09s/it]
2021-04-10 02:39:34,098 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 19/19 [00:07<00:00,  2.67it/s]
2021-04-10 02:39:42,843 - pytorch_modeler.py - INFO - Save best model
2021-04-10 02:39:42,844 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 19/19 [00:06<00:00,  2.73it/s]
2021-04-10 02:39:49,802 - pytorch_modeler.py - INFO - epoch:1/40, train_losses:14.340437456965446, val_source_losses:0.763328, val_target_losses:0.754760, best_flag:True
2021-04-10 02:39:49,803 - pytorch_modeler.py - INFO - train
100%|██████████| 189/189 [03:23<00:00,  1.08s/it]
2021-04-10 02:43:13,377 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 19/19 [00:06<00:00,  2.85it/s]
2021-04-10 02:43:21,637 - pytorch_modeler.py - INFO - Save best model
2021-04-10 02:43:21,638 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 19/19 [00:06<00:00,  2.85it/s]
2021-04-10 02:43:28,296 - pytorch_modeler.py - INFO - epoch:2/40, trai

2021-04-10 03:34:06,009 - pytorch_modeler.py - INFO - train
100%|██████████| 189/189 [03:23<00:00,  1.08s/it]
2021-04-10 03:37:29,489 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 19/19 [00:06<00:00,  2.85it/s]
2021-04-10 03:37:36,162 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 19/19 [00:06<00:00,  2.85it/s]
2021-04-10 03:37:42,823 - pytorch_modeler.py - INFO - epoch:17/40, train_losses:27.019843357556503, val_source_losses:1.183388, val_target_losses:1.422097, best_flag:False
2021-04-10 03:37:42,824 - pytorch_modeler.py - INFO - train
100%|██████████| 189/189 [03:23<00:00,  1.08s/it]
2021-04-10 03:41:06,302 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 19/19 [00:06<00:00,  2.85it/s]
2021-04-10 03:41:12,968 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 19/19 [00:06<00:00,  2.85it/s]
2021-04-10 03:41:19,640 - pytorch_modeler.py - INFO - epoch:18/40, train_losses:12.95415358914397, val_source_losses:0.681106, val_target_losses:0.6

100%|██████████| 189/189 [03:23<00:00,  1.08s/it]
2021-04-10 04:35:18,358 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 19/19 [00:06<00:00,  2.85it/s]
2021-04-10 04:35:25,018 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 19/19 [00:06<00:00,  2.85it/s]
2021-04-10 04:35:31,691 - pytorch_modeler.py - INFO - epoch:33/40, train_losses:17.84183120380476, val_source_losses:0.897975, val_target_losses:0.939044, best_flag:False
2021-04-10 04:35:31,692 - pytorch_modeler.py - INFO - train
100%|██████████| 189/189 [03:23<00:00,  1.08s/it]
2021-04-10 04:38:55,132 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 19/19 [00:06<00:00,  2.85it/s]
2021-04-10 04:39:01,794 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 19/19 [00:06<00:00,  2.85it/s]
2021-04-10 04:39:08,458 - pytorch_modeler.py - INFO - epoch:34/40, train_losses:17.794379504044755, val_source_losses:0.825208, val_target_losses:0.936546, best_flag:False
2021-04-10 04:39:08,459 - pytorch_mod

elapsed time: 8681.486120939 [sec]


In [12]:
machine_type = 'pump'
input_path = f'{OUT_FEATURE_DIR}/{machine_type}_features.pkl'
ext_data = pd.read_pickle(input_path)

FileNotFoundError: [Errno 2] No such file or directory: '/media/hiroki/HDD1TB/research/dcase2021_task2/output/CNN_finetune/extraction_features/pump_features.pkl'

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(ext_data['train']['features'], cmap='jet')
plt.colorbar()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(ext_data['valid_source']['features'], cmap='jet')
plt.colorbar()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(ext_data['train']['features'], cmap='jet')
plt.colorbar()

history = run('ToyCar')
with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    pickle.dump(history , file)

In [ ]:
print(M_means.shape)
plt.imshow(M_means, aspect='auto', cmap='jet')
plt.title(phase)
plt.colorbar()
plt.show()